In [ ]:
# FastAI version of your CNN image classification pipeline

from fastai.vision.all import *
import json

In [ ]:
# --- Assume you have images and labels as numpy arrays in shape (N, C, H, W) ---
# e.g. from load_data_npy() + preprocessing (transposition, normalization)

# Convert NumPy arrays to PyTorch tensors
X_train = torch.tensor(images_train).float()
y_train = torch.tensor(categories_train).long()

X_val = torch.tensor(images_val).float()
y_val = torch.tensor(categories_val).long()

X_test = torch.tensor(images_test).float()
y_test = torch.tensor(categories_test).long()

# Create Datasets
train_ds = Datasets(list(zip(X_train, y_train)), tfms=[[noop], [Categorize()]])
val_ds = Datasets(list(zip(X_val, y_val)), tfms=[[noop], [Categorize()]])

# Create DataLoaders
bs = 32
dls = DataLoaders.from_dsets(train_ds, val_ds, bs=bs, shuffle=True)

# --- Model definition ---
# You can use your custom model or FastAI built-in CNNs

class CNNExtended(nn.Module):
    def __init__(self):
        super(CNNExtended, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.pool1 = nn.MaxPool2d(2, 2)

        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.pool2 = nn.MaxPool2d(2, 2)

        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.bn3 = nn.BatchNorm2d(128)
        self.pool3 = nn.MaxPool2d(2, 2)

        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(128 * 18 * 18, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 6)

    def forward(self, x):
        x = self.pool1(F.relu(self.bn1(self.conv1(x))))
        x = self.pool2(F.relu(self.bn2(self.conv2(x))))
        x = self.pool3(F.relu(self.bn3(self.conv3(x))))
        x = self.flatten(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

# Create Learner
learn = Learner(dls, CNNExtended(), loss_func=CrossEntropyLossFlat(), metrics=accuracy)

# Train the model and save training history
learn.fit(10)

# Save the model
learn.save("cnn_extended_model")

# Save training history in TensorFlow-like JSON format
history_dict = {
    'train_loss': [float(loss) for loss in learn.recorder.losses],
    'valid_loss': [float(v[0]) for v in learn.recorder.values],
    'accuracy': [float(v[1]) if len(v) > 1 else None for v in learn.recorder.values],
    'metrics_names': learn.recorder.metric_names
}

with open("history_fastai.json", "w") as f:
    json.dump(history_dict, f)

print("Historial guardado en: history_fastai.json")

# Optional: plot training loss curve
learn.recorder.plot_loss()

# Evaluate on test set
test_dl = dls.test_dl(list(zip(X_test, y_test)))
results = learn.validate(dl=test_dl)
print(f"\nTest Loss: {results[0]:.4f}  |  Test Accuracy: {results[1]:.4f}")